In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [2]:
import pandas as pd

# Load only needed columns and a small sample
df = pd.read_csv(
    "combined_unique.csv",
    usecols=["Description", "Invoice", "Price"],  # load fewer columns
    encoding="latin1"
).dropna(subset=["Description", "Invoice", "Price"])

# Random 10000 invoices (or fewer if still large)
sample_invoices = df["Invoice"].drop_duplicates().sample(10000, random_state=42)
df_sample = df[df["Invoice"].isin(sample_invoices)].copy()


C:\Users\Minal\AppData\Local\Temp\ipykernel_8268\2651478795.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [3]:
# ---- Step 2: Deduplicate product descriptions ----
# (averaging the price for identical descriptions)
df_sample = (
    df_sample.groupby("Description", as_index=False)["Price"]
    .mean()
)

In [4]:
# clean text
df_sample["Description"] = (
    df_sample["Description"]
    .str.lower()
    .str.replace(r"[^a-z0-9 ]", " ", regex=True)
    .str.strip()
)


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Create TF-IDF vectors of descriptions
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(df_sample["Description"])

# Compute similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [6]:
# reset index to align rows with cosine_sim
df_sample = df_sample.reset_index(drop=True)

def recommend_items(product_name, n=5):
    # find matching product
    idx = df_sample[df_sample["Description"].str.contains(product_name.lower(), case=False, na=False)]
    if idx.empty:
        return "Product not found."

    idx = idx.index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i[0] for i in sim_scores[1:n+1]]

    return df_sample.iloc[top_indices][["Description", "Price"]]

# example: get 5 similar items
recommend_items("water bottle")


,Description,Price
2468,love hot water bottle,6.313028
961,chocolate hot water bottle,5.629439
3614,retrospot heart hot water bottle,5.478842
2153,hot water bottle babushka,5.462800
2152,hot stuff hot water bottle,4.306471
